In [ ]:
#| default_exp tagmaker

# Tag Maker
> Processor to quickly write ::: tags as comments

## Inspiration

As I was writing my new course, I found that having to do two sets of `:::` and something to put in between two `{}` annoying, so this is a quick processor to do that for me as a tag

In [ ]:
#| export
from nbdev.config import get_config
from nbdev.process import extract_directives
from nbdev.processors import Processor
from nbdev.export import nb_export
from nbdev.doclinks import nbglob
from nbdev.sync import write_nb

from fastcore.script import call_parse
from fastcore.xtras import Path

from string import Template

In [ ]:
#| hide
from fastcore.test import test_eq

In [ ]:
#| export
_LAYOUT_STR = Template("::: {$layout}\n$content\n")

In [ ]:
#| export
def convert_layout(cell, layout):
    "Parses cell formatted with ::: {$something}, and potentially :::"
    layout_format = layout[0]
    content = cell.source
    cell.source = _LAYOUT_STR.substitute(
        layout=layout,
        content=content
    )

In [ ]:
#| hide
from nbdev.processors import mk_cell, read_nb

In [ ]:
#| hide
cell = mk_cell("""#| layout .column-margin
A test""", "markdown")
cell.directives_ = extract_directives(cell, "#")
convert_layout(cell, cell.directives_["layout"][0])
test_eq(cell.source, """::: {.column-margin}
A test
""")

In [ ]:
#| export
class LayoutProc(Processor):
    """A proc that will automatically change #| layout format
    to ::: {format} ... :::
    """
    has_partial = False
    def cell(self, cell):
        if cell.cell_type == "markdown" and "layout" in cell.directives_:
            directives_ = cell.directives_["layout"]
            if self.has_partial and "end" in directives_:
                    cell.source = ":::"
                    self.has_partial = False
            else:
                convert_layout(cell, directives_[0])
                if "start" in directives_:
                    self.has_partial = True
                else:
                    cell.source += ":::"

In [ ]:
#| hide
from nbdev.process import NBProcessor, dict2nb

In [ ]:
#| hide
nb = {"cells":[
    mk_cell("""#| layout .column-margin
A test""", "markdown"),
    mk_cell("""#| layout .column-margin start
A test""", "markdown"),
    mk_cell("#| layout end", "markdown")
]}
processor = NBProcessor(procs=LayoutProc, nb=dict2nb(nb))
processor.process()
test_eq(processor.nb.cells[0].source, "::: {.column-margin}\nA test\n:::")
test_eq(processor.nb.cells[1].source, "::: {.column-margin}\nA test\n")
test_eq(processor.nb.cells[2].source, ":::")

An example usage of this proc is writing something such as (just the content):

In [ ]:
"""
#| layout .column-margin
Some test stuff!
""";

In a markdown cell and then running the processor

#| layout .column-margin
Some test stuff!

Or you can split it up into multiple cells by dictating `start` and `end`:

In [ ]:
"""
#| layout .column-margin start
Some test stuff!
""";

In [ ]:
"""
#| layout end
""";

#| layout .column-margin start
Some test stuff!

#| layout .column-margin end